In [1]:
%matplotlib qt
# %matplotlib inline

In [2]:
import numpy as np
import os
import h5py
import sys
import matplotlib.pyplot as plt

In [3]:
sys.path.append('/dls/science/groups/e02/Mohsen/code/Git_Repos/Staff-notebooks/ptyREX_sim_matrix')

In [4]:
from sim_utils import *
from sim_utils import _sigma
from recon_utils import *

In [5]:
matrix_path = '/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_matrix_ptyREX_25April2020'
# run_ID: 26042020 - 100 iterations 
# run_ID: 2000iter
# json_files = get_ptyREX_recon_list(matrix_path, run_id = '26042020') 
json_files = get_ptyREX_recon_list(matrix_path, run_id = '2000iter')

In [6]:
len(json_files)

112

In [7]:
# sorting the json files
conv_angles = [0.016, 0.020, 0.024, 0.030, 0.040, 0.050, 0.064, 0.084]
diff_overlap = [-22.5, 1.99, 18.33, 34.70, 51.00, 60.80, 69.37, 76.67]
real_probe_overlap = [-5, 2, 15, 35, 60, 70, 80]


focus_probes_dict = []
defocus_probes_dict = []
for file in json_files:
    j_dict = json_to_dict(file)
    if j_dict['experiment']['optics']['lens']['defocus'][0] == 0.00:
        for i, angle in enumerate(conv_angles):
            focus_probes_dict.append([])
            if j_dict['process']['common']['probe']['convergence'] == angle:
                focus_probes_dict[i].append(j_dict)
    else:
        for i, angle in enumerate(conv_angles):
            defocus_probes_dict.append([])
            if j_dict['process']['common']['probe']['convergence'] == angle:
                defocus_probes_dict[i].append(j_dict)
                
for i, angle in enumerate(conv_angles):
    #print(angle)
    focus_probes_dict[i].sort(key=lambda e: e['process']['common']['scan']['dR'][0], reverse = True)
    defocus_probes_dict[i].sort(key=lambda e: e['process']['common']['scan']['dR'][0], reverse = True)

In [40]:
# focused probe cases - fft only
rows = len(defocus_probes_dict[0])
cols = len(conv_angles)
#print(rows, cols)
fig, axs = plt.subplots(nrows=rows, ncols=cols, sharex=True, sharey=True,figsize=(8, 12))
#fig.suptitle('Focused probe conditions', fontsize = 12)

for idx in range(rows):    
      for idy in range(cols): 
            obj = crop_recon_obj(focus_probes_dict[idy][idx]['json_path'])
            #print(idx, idy)
            j_dict = j_dict = json_to_dict(focus_probes_dict[idy][idx]['json_path'])
            axs[idx,idy].imshow(np.log(abs(get_fft(obj))), cmap = 'viridis')
#             axs[idx,idy].imshow(abs(get_fft(obj)), cmap = 'viridis')

            axs[idx,idy].set_xticks([])
            axs[idx,idy].set_yticks([])
            if idx == 0:
                axs[idx,idy].set_title(str(1e3 * j_dict['process']['common']['probe']['convergence']) + 'mrad \n' + \
                    'step size ($\AA$) %2.2f'%(1e10*j_dict['process']['common']['scan']['dR'][0]) + '\n' + \
                                       str(diff_overlap[idy]) + '%overlap', color = 'red', fontsize = 8)
            elif idy == 0:
                axs[idx,idy].set_title('step size ($\AA$) %2.3f'%(1e10*j_dict['process']['common']['scan']['dR'][0]), color = 'black', fontsize = 5)
                axs[idx, idy].set_ylabel(str(real_probe_overlap[idx]) + '%overlap', color = 'red', fontsize = 8)
            else:
                axs[idx,idy].set_title('%2.3f'%(1e10*j_dict['process']['common']['scan']['dR'][0]), color = 'black', fontsize = 5)
#plt.tight_layout()

In [41]:
# focused probe cases - obj phase only
rows = len(defocus_probes_dict[0])
cols = len(conv_angles)
#print(rows, cols)
fig, axs = plt.subplots(nrows=rows, ncols=cols, sharex=True, sharey=True,figsize=(18, 10))
#fig.suptitle('Focused probe conditions', fontsize = 12)

for idx in range(rows):    
      for idy in range(cols): 
            obj = crop_recon_obj(focus_probes_dict[idy][idx]['json_path'])
            #print(idx, idy)
            j_dict = j_dict = json_to_dict(focus_probes_dict[idy][idx]['json_path'])
            
            img = abs(np.min(np.angle(obj))) + np.angle(obj)
            sh0 = img.shape[0]
            sh1 = 82
            img_crop = img[int(sh0/2 - sh1/2):int(sh0/2 + sh1/2), int(sh0/2 - sh1/2):int(sh0/2 + sh1/2)]
            
            axs[idx,idy].imshow(img_crop, cmap = 'magma_r')
#             axs[idx,idy].imshow(abs(get_fft(obj)), cmap = 'viridis')

            axs[idx,idy].set_xticks([])
            axs[idx,idy].set_yticks([])
            if idx == 0:
                axs[idx,idy].set_title(str(1e3 * j_dict['process']['common']['probe']['convergence']) + 'mrad \n' + \
                    'step size ($\AA$) %2.2f'%(1e10*j_dict['process']['common']['scan']['dR'][0]) + '\n' + \
                                       str(diff_overlap[idy]) + '%overlap', color = 'red', fontsize = 8)
            elif idy == 0:
                axs[idx,idy].set_title('step size ($\AA$) %2.3f'%(1e10*j_dict['process']['common']['scan']['dR'][0]), color = 'black', fontsize = 8)
                axs[idx, idy].set_ylabel(str(real_probe_overlap[idx]) + '%overlap', color = 'red', fontsize = 8)
            else:
                axs[idx,idy].set_title('%2.3f'%(1e10*j_dict['process']['common']['scan']['dR'][0]), color = 'black', fontsize = 6)
#plt.tight_layout()

In [42]:
# defocused probe cases - fft only
rows = len(defocus_probes_dict[0])
cols = len(conv_angles)
#print(rows, cols)
fig, axs = plt.subplots(nrows=rows, ncols=cols, sharex=True, sharey=True,figsize=(8, 11))
for idx in range(rows):    
      for idy in range(cols): 
            try:
                obj = crop_recon_obj(defocus_probes_dict[idy][idx]['json_path'])
                #print(idx, idy)
                j_dict = j_dict = json_to_dict(defocus_probes_dict[idy][idx]['json_path'])
                axs[idx,idy].imshow(np.sqrt(abs(get_fft(obj))), cmap = 'viridis')
                axs[idx,idy].set_xticks([])
                axs[idx,idy].set_yticks([])
                if idx == 0:
                    axs[idx,idy].set_title(str(1e3 * j_dict['process']['common']['probe']['convergence']) + 'mrad \n' + \
                        'step size ($\AA$) %2.3f'%(1e10*j_dict['process']['common']['scan']['dR'][0])+ '\n' + \
                                       str(diff_overlap[idy]) + '%overlap', color = 'red', fontsize = 8)
                elif idy == 0:
                    axs[idx,idy].set_title('step size ($\AA$) %2.3f'%(1e10*j_dict['process']['common']['scan']['dR'][0]), color = 'black', fontsize = 5)
                    axs[idx, idy].set_ylabel(str(real_probe_overlap[idx]) + '%overlap', color = 'red', fontsize = 8)
                else:
                    axs[idx,idy].set_title('%2.3f'%(1e10*j_dict['process']['common']['scan']['dR'][0]), color = 'black', fontsize = 5)
            except IndexError:
                pass
#plt.tight_layout()            

In [43]:
# defocused probe cases - obj phase only
rows = len(defocus_probes_dict[0])
cols = len(conv_angles)
#print(rows, cols)
fig, axs = plt.subplots(nrows=rows, ncols=cols, sharex=True, sharey=True,figsize=(8, 11))
for idx in range(rows):    
      for idy in range(cols): 
            try:
                obj = crop_recon_obj(defocus_probes_dict[idy][idx]['json_path'])
                #print(idx, idy)
                j_dict = json_to_dict(defocus_probes_dict[idy][idx]['json_path'])
                
                img = abs(np.min(np.angle(obj))) + np.angle(obj)
                sh0 = img.shape[0]
                sh1 = 82
                img_crop = img[int(sh0/2 - sh1/2):int(sh0/2 + sh1/2), int(sh0/2 - sh1/2):int(sh0/2 + sh1/2)]
                
                axs[idx,idy].imshow(img_crop, cmap = 'magma_r')
                axs[idx,idy].set_xticks([])
                axs[idx,idy].set_yticks([])
                if idx == 0:
                    axs[idx,idy].set_title(str(1e3 * j_dict['process']['common']['probe']['convergence']) + 'mrad \n' + \
                        'step size ($\AA$) %2.3f'%(1e10*j_dict['process']['common']['scan']['dR'][0])+ '\n' + \
                                       str(diff_overlap[idy]) + '%overlap', color = 'red', fontsize = 8)
                elif idy == 0:
                    axs[idx,idy].set_title('step size ($\AA$) %2.3f'%(1e10*j_dict['process']['common']['scan']['dR'][0]), color = 'black', fontsize = 5)
                    axs[idx, idy].set_ylabel(str(real_probe_overlap[idx]) + '%overlap', color = 'black', fontsize = 5)
                else:
                    axs[idx,idy].set_title('%2.3f'%(1e10*j_dict['process']['common']['scan']['dR'][0]), color = 'black', fontsize = 5)
            except IndexError:
                pass
#plt.tight_layout()            

# Comparison with sim potential

In [8]:
# we get one of the potentials as ground truth to compare the recons with.
focus_probes_dict[4][5]['sim_path']

'/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_matrix_ptyREX_25April2020/Graphene_defect_20.0mrad_0.00A_def_1.70A_step_size/Graphene_defect_20.0mrad_0.00A_def_1.70A_step_size.h5'

In [9]:
pot = get_potential(focus_probes_dict[4][5]['sim_path'])

In [10]:
sh = pot.shape[0]
obj_pot = pot[int(0.33*sh):int(0.66*sh), int(0.33*sh):int(0.66*sh)]
obj_pot_fft = np.fft.fftshift(np.fft.fft2(obj_pot))
phase_ideal = _sigma(80000) * obj_pot
fig, ax = plt.subplots(1,3,figsize=(11,4))
im = ax[0].imshow(obj_pot)
fig.colorbar(im, ax = ax[0])
ax[1].imshow(np.log(abs(obj_pot_fft)), cmap = 'viridis')
im2 = ax[2].imshow(phase_ideal)
fig.colorbar(im2, ax = ax[2])
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[1].set_xticks([])
ax[1].set_yticks([])
ax[2].set_xticks([])
ax[2].set_yticks([])
ax[0].set_title('object potential in V$\AA$')
ax[1].set_title('fft of object potential')
ax[2].set_title('ideal phase shift')
# plt.tight_layout()            

Text(0.5, 1.0, 'ideal phase shift')

In [11]:
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import normalized_root_mse as nrmse
from skimage.metrics import mean_squared_error as mse
from skimage.feature import register_translation
from scipy.ndimage import fourier_shift


In [12]:
# binning the ideal phase by 2 - the sim pixelSize is half the recn pixelSize - due to the way 4DSTEM
# data is saved in pyprimsatic
phase_ideal_hs = hs.signals.Signal2D(phase_ideal)
phase_ideal_bin = phase_ideal_hs.rebin(scale = (2,2))
phase_ideal_bin = phase_ideal_bin.data

In [49]:
offset_img_r = np.real(offset_img)

In [50]:
ratio = np.divide(offset_img_r, phase_ideal_bin)

In [51]:
plt.figure()
plt.imshow(ratio, cmap = 'viridis')
plt.colorbar()
ratio_mean = np.mean(ratio)

In [52]:
ratio_mean

0.9884089493296008

In [13]:
ssim_scores = []
nrmse_scores = []
mse_scores = []
case_num = 0
convergence = []
step_size = []
for i in range(len(conv_angles)):
    for n in range(len(defocus_probes_dict[i])):
        obj = crop_recon_obj(defocus_probes_dict[i][n]['json_path'])
        convergence.append(defocus_probes_dict[i][n]['process']['common']['probe']['convergence'])
        step_size.append(defocus_probes_dict[i][n]['process']['common']['scan']['dR'][0])
        img = abs(np.min(np.angle(obj))) + np.angle(obj)
        sh0 = img.shape[0]
        sh1 = phase_ideal_bin.shape[0]
        img_crop = img[int(sh0/2 - sh1/2):int(sh0/2 + sh1/2), int(sh0/2 - sh1/2):int(sh0/2 + sh1/2)]
        shift, error, diffphase = register_translation(phase_ideal_bin, img_crop)
        offset_img = fourier_shift(np.fft.fftn(img_crop), shift)
        offset_img = np.fft.ifftn(offset_img)
        ssim_score = ssim(phase_ideal_bin, offset_img)
        nrmse_score = nrmse(phase_ideal_bin, offset_img, normalization= 'min-max')
        mse_score = mse(phase_ideal_bin, offset_img)
        ssim_scores.append(ssim_score)
        nrmse_scores.append(nrmse_score)
        mse_scores.append(mse_score)

convergence = np.asarray(convergence)
fig, ax = plt.subplots(1,2,figsize=(11,4))
for angle in conv_angles:
    inds = np.where(convergence==angle)
    _steps = np.take(step_size, inds[0])
    _ssim = np.take(ssim_scores, inds[0])
    _nrmse = np.take(nrmse_scores, inds[0])
    _mse = np.take(mse_scores, inds[0])

    ax[0].plot(_steps, _ssim,label = str(angle))
    ax[0].set_title('ssim scores')
    ax[0].set_xlabel('step_size(m)')
    ax[1].plot(_steps, _nrmse,label = str(angle))
    ax[1].set_title('nrmse scores')
    ax[1].set_xlabel('step_size(m)')
    ax[0].legend()
    ax[1].legend()
fig.suptitle('ssim and nrmse scores as function of convergence angle \n defocused probe cases')

/dls_sw/apps/python/anaconda/4.6.14/64/envs/epsic3.7/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im1.dtype.
/dls_sw/apps/python/anaconda/4.6.14/64/envs/epsic3.7/lib/python3.7/site-packages/skimage/metrics/_structural_similarity.py:177: ComplexWarning: Casting complex values to real discards the imaginary part
  im2 = im2.astype(np.float64)
/dls_sw/apps/python/anaconda/4.6.14/64/envs/epsic3.7/lib/python3.7/site-packages/numpy/core/_methods.py:151: ComplexWarning: Casting complex values to real discards the imaginary part
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/dls_sw/apps/python/anaconda/4.6.14/64/envs/epsic3.7/lib/python3.7/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/dls_sw/apps/python/anaconda/4.6.14/64/envs/epsic3.7/lib/python3.7/site-packages/numpy/core/_asarray.py:85: Co

Text(0.5, 0.98, 'ssim and nrmse scores as function of convergence angle \n defocused probe cases')

In [54]:
ssim_scores = []
nrmse_scores = []
mse_scores = []
case_num = 0
convergence = []
step_size = []
for i in range(6):
    for n in range(len(focus_probes_dict[i])):
        obj = crop_recon_obj(focus_probes_dict[i][n]['json_path'])
        convergence.append(focus_probes_dict[i][n]['process']['common']['probe']['convergence'])
        step_size.append(focus_probes_dict[i][n]['process']['common']['scan']['dR'][0])
        img = abs(np.min(np.angle(obj))) + np.angle(obj)
        sh0 = img.shape[0]
        sh1 = phase_ideal_bin.shape[0]
        img_crop = img[int(sh0/2 - sh1/2):int(sh0/2 + sh1/2), int(sh0/2 - sh1/2):int(sh0/2 + sh1/2)]
        shift, error, diffphase = register_translation(phase_ideal_bin, img_crop)
        offset_img = fourier_shift(np.fft.fftn(img_crop), shift)
        offset_img = np.fft.ifftn(offset_img)
        ssim_score = ssim(phase_ideal_bin, offset_img)
        nrmse_score = nrmse(phase_ideal_bin, offset_img, normalization= 'min-max')
        mse_score = mse(phase_ideal_bin, offset_img)
        ssim_scores.append(ssim_score)
        nrmse_scores.append(nrmse_score)
        mse_scores.append(mse_score)

convergence = np.asarray(convergence)
fig, ax = plt.subplots(1,2,figsize=(11,4))
for angle in conv_angles:
    inds = np.where(convergence==angle)
    _steps = np.take(step_size, inds[0])
    _ssim = np.take(ssim_scores, inds[0])
    _nrmse = np.take(nrmse_scores, inds[0])
    _mse = np.take(mse_scores, inds[0])

    ax[0].plot(_steps, _ssim,label = str(angle))
    ax[0].set_title('ssim scores')
    ax[0].set_xlabel('step_size(m)')
    ax[1].plot(_steps, _nrmse,label = str(angle))
    ax[1].set_title('nrmse scores')
    ax[1].set_xlabel('step_size(m)')
    ax[0].legend()
    ax[1].legend()
fig.suptitle('ssim and nrmse scores as function of convergence angle \n focused probe cases')
None

/dls_sw/apps/python/anaconda/4.6.14/64/envs/epsic3.7/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im1.dtype.
/dls_sw/apps/python/anaconda/4.6.14/64/envs/epsic3.7/lib/python3.7/site-packages/skimage/metrics/_structural_similarity.py:177: ComplexWarning: Casting complex values to real discards the imaginary part
  im2 = im2.astype(np.float64)
/dls_sw/apps/python/anaconda/4.6.14/64/envs/epsic3.7/lib/python3.7/site-packages/numpy/core/_methods.py:151: ComplexWarning: Casting complex values to real discards the imaginary part
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/dls_sw/apps/python/anaconda/4.6.14/64/envs/epsic3.7/lib/python3.7/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/dls_sw/apps/python/anaconda/4.6.14/64/envs/epsic3.7/lib/python3.7/site-packages/numpy/core/_asarray.py:85: Co

In [62]:
# To plot objects as a function of convergence angles
conv_angles = np.asarray(conv_angles)
angle = 0.084
ind = np.where(conv_angles==angle)

objects = []
step_sizes = []
sh1 = 84
for json_dict in defocus_probes_dict[ind[0][0]]:
    step_size = json_dict['process']['common']['scan']['dR'][0]
    step_sizes.append(step_size)
    obj = crop_recon_obj(json_dict['json_path'])
    img = abs(np.min(np.angle(obj))) + np.angle(obj)
    sh0 = img.shape[0]
    img_crop = img[int(sh0/2 - sh1/2):int(sh0/2 + sh1/2), int(sh0/2 - sh1/2):int(sh0/2 + sh1/2)]
    objects.append(img_crop)
fig, ax = plt.subplots(4,2, figsize=(8,11))
for i, step in enumerate(step_sizes):

    ax[int(i / 2), i%2].imshow(objects[i], cmap = 'magma_r')
    ax[int(i / 2), i%2].set_title('%2.3f'%(1e10*step)+' $\AA$' )

    ax[int(i / 2), i%2].set_xticks([])
    ax[int(i / 2), i%2].set_yticks([])
fig.suptitle('probe convergence ' + str(angle)+ ' rad')
None

In [32]:
conv_angles

array([0.016, 0.02 , 0.024, 0.03 , 0.04 , 0.05 , 0.064, 0.084])

In [107]:
# To plot errors function of convergence angles
conv_angles = np.asarray(conv_angles)
angle = 0.016
ind = np.where(conv_angles==angle)

errors = []
step_sizes = []
for json_dict in focus_probes_dict[ind[0][0]]:
    step_size = json_dict['process']['common']['scan']['dR'][0]
    step_sizes.append(step_size)
    name, ext = os.path.splitext(json_dict['json_path'])
    error = get_error(name + '.hdf')
    errors.append(error)
fig, ax = plt.subplots(4,2, figsize=(8,12))
for i, step in enumerate(step_sizes):

    ax[int(i / 2), i%2].plot(errors[i])
    ax[int(i / 2), i%2].set_title('%2.3f'%(1e10*step)+'$\AA$' )

fig.suptitle('error vs iter num ' + str(angle)+ 'rad')
None

# Evaluation using atomap

In [14]:
%matplotlib qt
import atomap.api as am

In [15]:
# Getting the reference positions
ref_hs = hs.signals.Signal2D(phase_ideal_bin)
ref_hs_crop = ref_hs.isig[5:70,5:70]
ref_atom_positions = am.get_atom_positions(ref_hs_crop, separation=2)
ref_sublattice = am.Sublattice(ref_atom_positions, image=ref_hs_crop.data)
ref_sublattice.find_nearest_neighbors()
ref_sublattice.refine_atom_positions_using_center_of_mass()
ref_sublattice.refine_atom_positions_using_2d_gaussian()
ref_atom_list = ref_sublattice.atom_list

Gaussian fitting: 100%|██████████| 149/149 [00:11<00:00, 13.12it/s]


In [16]:
ref_atom_list[0]

<Atom_Position,  (x:61.7,y:61.1,sx:0.7,sy:0.6,r:1.9,e:1.2)>

In [17]:
# This is not returning the same value as e!
ref_atom_list[0].amplitude_gaussian 

1.338411804525878

In [18]:
ref_atom_list[0].pixel_x

61.694970982469904

In [19]:
ref_atom_list[0].sigma_average

0.6677949193333059

In [20]:
ref_coord = []
for i in range(len(ref_atom_list)):
    
    ref_coord.append(list((ref_atom_list[i].pixel_x,
                    ref_atom_list[i].pixel_y,
                    ref_atom_list[i].sigma_average,
                    ref_atom_list[i].amplitude_gaussian)))

ref_coord = np.asarray(ref_coord)
print('Number of atoms in reference: ', ref_coord.shape[0])


Number of atoms in reference:  149


In [22]:
# Getting the experimental positions and comparison


In [23]:
# defocused probe cases 
rows = len(defocus_probes_dict[0])
cols = len(conv_angles)

for idx in range(rows):    
      for idy in range(cols): 
            obj = crop_recon_obj(defocus_probes_dict[idy][idx]['json_path'])
            img = abs(np.min(np.angle(obj))) + np.angle(obj)
            sh0 = img.shape[0]
            sh1 = phase_ideal_bin.shape[0]
            img_crop = img[int(sh0/2 - sh1/2):int(sh0/2 + sh1/2), int(sh0/2 - sh1/2):int(sh0/2 + sh1/2)]
            shift, error, diffphase = register_translation(phase_ideal_bin, img_crop)
            offset_img = fourier_shift(np.fft.fftn(img_crop), shift)
            offset_img = np.real(np.fft.ifftn(offset_img))

            img_hs = hs.signals.Signal2D(offset_img)
            img_hs_crop = img_hs.isig[5:70,5:70]


            exp_positions = am.get_atom_positions(img_hs_crop, separation=2)
            exp_sublattice = am.Sublattice(exp_positions, image=img_hs_crop.data)

            exp_sublattice.find_nearest_neighbors()
            exp_sublattice.refine_atom_positions_using_center_of_mass()
            exp_sublattice.refine_atom_positions_using_2d_gaussian()

            exp_atom_list = exp_sublattice.atom_list


            exp_coord = []
            for i in range(len(exp_atom_list)):

                exp_coord.append(list((exp_atom_list[i].pixel_x,
                                exp_atom_list[i].pixel_y,
                                exp_atom_list[i].sigma_average,
                                exp_atom_list[i].amplitude_gaussian)))

            exp_coord = np.asarray(exp_coord)
            defocus_probes_dict[idy][idx]['atom_pos'] = exp_coord

          

Gaussian fitting: 100%|██████████| 146/146 [00:18<00:00,  7.69it/s]


In [24]:
# identified atom positions versus the known positions
rows = len(defocus_probes_dict[0])
cols = len(conv_angles)
#print(rows, cols)
fig, axs = plt.subplots(nrows=rows, ncols=cols, sharex=True, sharey=True,figsize=(8, 11))
for idx in range(rows):    
      for idy in range(cols): 
            try:
                j_dict = json_to_dict(defocus_probes_dict[idy][idx]['json_path'])
                exp_coord = defocus_probes_dict[idy][idx]['atom_pos']
#                 print(json_dict['json_path'].split('/')[-2])
#                 print('Difference in number f atoms: ', ref_coord.shape[0] - exp_coord.shape[0])
                
                missed_atoms = ref_coord.shape[0] - exp_coord.shape[0]
                
                axs[idx,idy].scatter(ref_coord[:,0], ref_coord[:,1], s = 1, c = 'g')
                axs[idx,idy].scatter(exp_coord[:,0], exp_coord[:,1], s = 1, c = 'r')
                axs[idx,idy].set_xticks([])
                axs[idx,idy].set_yticks([])
                axs[idx,idy].set_title(missed_atoms, color = 'black')
#                 if idx == 0:
#                     axs[idx,idy].set_title(str(1e3 * j_dict['process']['common']['probe']['convergence']) + 'mrad \n' + \
#                         'step size ($\AA$) %2.3f'%(1e10*j_dict['process']['common']['scan']['dR'][0])+ '\n' + \
#                                        str(diff_overlap[idy]) + '%overlap', color = 'black', fontsize = 8)
#                 elif idy == 0:
#                     axs[idx,idy].set_title('step size ($\AA$) %2.3f'%(1e10*j_dict['process']['common']['scan']['dR'][0]), color = 'black', fontsize = 5)
#                     axs[idx, idy].set_ylabel(str(real_probe_overlap[idx]) + '%overlap', color = 'black', fontsize = 5)
#                 else:
#                     axs[idx,idy].set_title('%2.3f'%(1e10*j_dict['process']['common']['scan']['dR'][0]), color = 'black', fontsize = 5)
            except IndexError:
                pass
#plt.tight_layout()            

In [31]:
plt.figure()
plt.scatter(ref_coord[:,0], ref_coord[:,1], s = 1, c = 'g')
exp_pos = defocus_probes_dict[0][6]['atom_pos']
plt.scatter(exp_pos[:,0], exp_pos[:,1], s = 1, c = 'r')

In [56]:
results_folder = '/dls/science/groups/e02/Mohsen/code/Git_Repos/Staff-notebooks/ptyREX_sim_matrix/fitting_results'

rows = len(defocus_probes_dict[0])
cols = len(conv_angles)
for idx in range(rows):    
      for idy in range(cols): 
            h5_file_name = os.path.join(results_folder, defocus_probes_dict[idy][idx]['json_path'].split('/')[-2] + '.h5')
            save_dict_to_hdf5(defocus_probes_dict[idy][idx], h5_file_name)

In [57]:
dd = load_dict_from_hdf5('/dls/science/groups/e02/Mohsen/code/Git_Repos/Staff-notebooks/ptyREX_sim_matrix/fitting_results/Graphene_defect_32.0mrad_162.50A_def_14.14A_step_size.h5')
print(dd)

{'atom_pos': array([[47.42116222, 62.10907669,  0.82690293,  0.43033646],
       [60.18042943, 60.76739387,  1.57025419,  1.01218751],
       [ 6.27884185, 60.57151211,  1.64482942,  1.39652989],
       [50.86517316, 59.87881546,  0.88497388,  0.35926475],
       [43.42158049, 59.82519518,  1.44730321,  1.17415328],
       [33.86887706, 59.76730307,  1.61413313,  1.79769618],
       [21.0350584 , 59.59644409,  0.65761316,  0.6808318 ],
       [18.25126214, 59.63363769,  0.44453795,  0.28740539],
       [13.1216083 , 59.70504054,  1.28972242,  1.67355149],
       [57.00897678, 58.95348187,  1.        ,  0.        ],
       [28.09070054, 58.92356827,  1.19368873,  0.66525146],
       [53.65101126, 58.11942771,  0.5305444 ,  0.34074303],
       [45.77654869, 57.29913357,  0.48256974,  0.1162576 ],
       [36.00223391, 56.54303876,  0.68151732,  0.6580981 ],
       [57.98856436, 55.99212518,  1.        ,  0.        ],
       [16.88504168, 56.11591564,  1.        ,  0.        ],
       [11.

/dls_sw/apps/python/anaconda/4.6.14/64/envs/epsic3.7/lib/python3.7/site-packages/ipykernel_launcher.py:38: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.


In [49]:
# Saving the atomic fitting data
import numpy as np
import h5py
import os
def save_dict_to_hdf5(dic, filename):
    """
    ....
    """
    if os.path.exists(filename):
        with h5py.File(filename, 'a') as h5file:
            recursively_save_dict_contents_to_group(h5file, '/', dic)
    else:
        
        with h5py.File(filename, 'w') as h5file:
            recursively_save_dict_contents_to_group(h5file, '/', dic)

def recursively_save_dict_contents_to_group(h5file, path, dic):
    """
    ....
    """
    for key, item in dic.items():
        if isinstance(item, (np.ndarray, list, float, int, str)):
            h5file[path + key] = item
        elif isinstance(item, dict):
            recursively_save_dict_contents_to_group(h5file, path + key + '/', item)


# Trials on single datasets

In [25]:
ref_hs = hs.signals.Signal2D(phase_ideal_bin)
ref_hs_crop = ref_hs.isig[5:70,5:70]
ref_hs_crop.plot()

In [36]:
s_peaks = am.get_feature_separation(ref_hs_crop, separation_range=(2, 20))

100%|██████████| 280/280 [00:00<00:00, 3478.03it/s]


In [37]:
s_peaks.plot()

In [236]:
ref_atom_positions = am.get_atom_positions(ref_hs_crop, separation=2)

In [237]:
ref_sublattice = am.Sublattice(ref_atom_positions, image=ref_hs_crop.data)

In [238]:
ref_sublattice

<Sublattice,  (atoms:149,planes:0)>

In [239]:
ref_sublattice.find_nearest_neighbors()
ref_sublattice.refine_atom_positions_using_center_of_mass()
ref_sublattice.refine_atom_positions_using_2d_gaussian()

Gaussian fitting: 100%|██████████| 149/149 [00:10<00:00, 14.47it/s]


In [240]:
ref_sublattice.get_position_history().plot()

100%|██████████| 149/149 [00:00<00:00, 2943.29it/s]


In [241]:
ref_atom_list = ref_sublattice.atom_list

In [242]:
ref_atom_list[0]

<Atom_Position,  (x:61.7,y:61.1,sx:0.7,sy:0.6,r:1.9,e:1.2)>

In [270]:
defocus_probes_dict[0][6]['json_path']

'/dls/e02/data/2020/cm26481-1/processing/pty_simulated_data_MD/sim_matrix_ptyREX_25April2020/Graphene_defect_8.0mrad_281.70A_def_4.35A_step_size/2000iter_20200426-222252.json'

In [218]:
plt.figure()
plt.imshow(np.angle(test_obj))

In [271]:
test_obj = crop_recon_obj(defocus_probes_dict[0][6]['json_path'])

img = abs(np.min(np.angle(test_obj))) + np.angle(test_obj)
sh0 = img.shape[0]
sh1 = phase_ideal_bin.shape[0]
img_crop = img[int(sh0/2 - sh1/2):int(sh0/2 + sh1/2), int(sh0/2 - sh1/2):int(sh0/2 + sh1/2)]
shift, error, diffphase = register_translation(phase_ideal_bin, img_crop)
offset_img = fourier_shift(np.fft.fftn(img_crop), shift)
offset_img = np.real(np.fft.ifftn(offset_img))

fig, ax = plt.subplots(1,2)
ax[0].imshow(img_crop)
ax[1].imshow(np.real(offset_img))

img_hs = hs.signals.Signal2D(offset_img)
img_hs_crop = img_hs.isig[5:70,5:70]
img_hs_crop.plot()

In [231]:
img_hs.plot()

In [272]:
s_peaks = am.get_feature_separation(img_hs, separation_range=(2, 20))
s_peaks.plot()

100%|██████████| 188/188 [00:00<00:00, 3748.05it/s]


In [273]:
test_positions = am.get_atom_positions(img_hs_crop, separation=2)

In [274]:
test_sublattice = am.Sublattice(test_positions, image=img_hs_crop.data)

In [275]:
test_sublattice

<Sublattice,  (atoms:104,planes:0)>

In [276]:
test_sublattice.find_nearest_neighbors()
test_sublattice.refine_atom_positions_using_center_of_mass()
test_sublattice.refine_atom_positions_using_2d_gaussian()

Gaussian fitting: 100%|██████████| 104/104 [00:13<00:00,  7.64it/s]


In [184]:
test_sublattice.get_position_history().plot()

100%|██████████| 239/239 [00:00<00:00, 1313.30it/s]


In [65]:
test_sublattice.plot_ellipticity_map()

In [277]:
test_atom_list = test_sublattice.atom_list

In [278]:
test_sublattice.plot()

In [279]:
test_atom_list[0]

<Atom_Position,  (x:38.6,y:61.8,sx:1.6,sy:2.2,r:0.4,e:1.4)>

In [280]:
ref_coord = []
test_coord = []
for i in range(len(ref_atom_list)):
    ref_coord.append(list(ref_atom_list[i].get_pixel_position()))
for i in range(len(test_atom_list)):
    test_coord.append(list(test_atom_list[i].get_pixel_position()))
ref_coord = np.asarray(ref_coord)
test_coord = np.asarray(test_coord)

In [281]:
plt.figure()
plt.scatter(ref_coord[:,0], ref_coord[:,1])
plt.scatter(test_coord[:,0], test_coord[:,1])

In [282]:
def atom_dist(x1, y1, x2, y2):
    return(np.sqrt((y2-y1)**2+(x2-x1)**2))


def check_atom_found(ref_atom, exp_list, tol):
    distance_check = []
    for exp_atom in exp_list:
        distance_check.append(atom_dist(ref_atom[0], ref_atom[1], exp_atom[0], exp_atom[1]) < tol)
    return any(distance_check)

In [283]:
len(ref_coord)

149

In [284]:
len(test_coord)

104

In [293]:
ref_to_compare = []
ind_to_del = []
match_count = 0
for i, atom in enumerate(ref_coord):
    if check_atom_found(atom, test_coord, 2.28):
        match_count += 1
    else: 
        ind_to_del.append(i)
ref_to_compare = np.delete(ref_coord, ind_to_del, 0)
print('number of atoms missing: ', len(ref_coord) - match_count)

number of atoms missing:  45


In [294]:
match_count

104

In [295]:
len(ref_coord) - len(test_coord)

45

In [296]:
len(ref_to_compare)

104

In [297]:
mse = ((ref_to_compare - test_coord)**2).mean()
print(mse)

279.6274438286989
